# Limpieza de Datos

En la presente sección se procede a limpiar los datos brutos de la lectora óptica. Para poder visualizar mejor el trabajo a
continuación se presentan el formato *limpio* de los datos y se compara con una muestra de los datos brutos.

## Aspecto de los datos

Idealmente para un proceso adecuado, los datos deberían tener el siguiente aspecto

![](https://github.com/kamecon/TFM_Kschool/raw/master/data1.jpg)

En este caso, se podrían cargar los datos en un data frame o incluso una hoja de cálculo sin problemas. No obstante los datos
brutos suelen tener este aspecto

![](https://github.com/kamecon/TFM_Kschool/raw/master/data2.jpg)

La anterior es una muestra sesgada de los datos para precisamente mostrar los elementos a limpiar, pero no es una muestra
exhaustiva de los errores que nos podemos encontrar. Dichos errores son una combinación de errores humanos y de lectura de la lectora óptica. A continuación se enumeran algunos:

* Los usuarios dejan casillas en blanco
* Colocan mal alguno de los códigos
* No rellenan bien la casilla
* La lectora desplaza algunos campos
* La lectora interpreta mal algún código y lanza un símbolo en lugar del número
* etc 

Dado que parte de estos errores se corregían de forma manual, afortunadamente tenemos una amplia compresión de su orígen y
solución


## Proceso

Se importan las librerías necesarias (de momento)

In [5]:
import pandas as pd

In [6]:
import numpy as np

Se cargan los datos

In [7]:
encu_1 = pd.read_csv('encuesta.txt', sep=';', skiprows=1, header=None)
encu_1.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,2,1,A,0 2,0 6,05,06,04,08,07,03,05,07,07,09,,,,,NaN
1,2,1,A,00,006,07,07,06,06,06,08,07,,08,08,,,,,NaN
2,2,1,A,002,006,10,10,10,10,10,10,10,10,10,10,,,,,NaN
3,2,1,A,002,,08,07,07,06,07,08,07,,07,08,,,,,NaN
4,2,1,A,002,006,09,08,08,08,09,09,08,09,08,08,,,,,NaN


Se Cogen los items de respuesta (columnas de la 5 hasta el final) y con una expresión regular se sustituyen los espacios en blanco por NaN y luego se eliminan las columnas donde todos los valores son NaN's

In [8]:
encu_2 = encu_1.iloc[:,5:].replace(r'\s+', np.nan, regex=True).dropna(axis=1, how='all')
encu_2.head()

,5,6,7,8,9,10,11,12,13,14,15,16
0,05,06,04,08,07,03,05,07,07,09,NaN,NaN
1,07,07,06,06,06,08,07,NaN,08,08,NaN,NaN
2,10,10,10,10,10,10,10,10,10,10,NaN,NaN
3,08,07,07,06,07,08,07,NaN,07,08,NaN,NaN
4,09,08,08,08,09,09,08,09,08,08,NaN,NaN


Obtenemos la primera versión reducida del proceso de limpieza. Nótese que estamos trabajando solo con los items de respuesta.

En este paso se deben realizar varias operaciones:

* Solo hay 10 items de respuesta (columnas 5 a 9), pero una vez eliminadas las columnas con todos los valores iguales a NaN aun
  quedan 2 columnas adicionales. Esto es debido a que la lectora ha desplazado algunas calificaciones hacia la derecha. Asi que 
  lo primero que haremos es corregir ese desplazamiento.
  
* Existen items sin calificar, a estos se le imputa el valor mediano del resto de items

Comenzamos con la primera operación, corregir el desplazamiento realizado por la lectora.

Se localizan los NaN's de la penúltima columna

In [9]:
i_nulos_p = encu_2.iloc[:,-2].isnull()

Lo anterior nos da un vector booleano de unos y ceros, los unos (TRUE) nos dan los elementos nulos.

En la próxima celda localizamos los valores no nulos buscando (con where) aquellas celdas iguales a cero

In [10]:
i_nonulos_p = np.where(i_nulos_p == 0)[0].tolist()
i_nonulos_p

[87,
 175,
 204,
 206,
 246,
 392,
 548,
 990,
 1269,
 1493,
 2000,
 2193,
 2480,
 2484,
 2563,
 2753,
 3035,
 3125,
 3231,
 3259,
 3709]

Repetimos lo anterior con la última columna

In [11]:
i_nulos_u = encu_2.iloc[:,-1].isnull()
i_nonulos_u = np.where(i_nulos_u == 0)[0].tolist()
i_nonulos_u

[175, 183, 302]

Se construye una lista que contenga los no nulos de la última y penúltima fila, usando la unión (``|``)

In [12]:
i_nonulos_temp = set(i_nonulos_p) | set(i_nonulos_u)
i_nonulos = sorted(list(i_nonulos_temp))
i_nonulos

[87,
 175,
 183,
 204,
 206,
 246,
 302,
 392,
 548,
 990,
 1269,
 1493,
 2000,
 2193,
 2480,
 2484,
 2563,
 2753,
 3035,
 3125,
 3231,
 3259,
 3709]

Se comprueba que la longitud es distinta a los nonulos de las dos últimas columnas

In [15]:
len(i_nonulos_p), len(i_nonulos_u), len(i_nonulos)

(21, 3, 23)

In [16]:
encu_2.iloc[i_nonulos,:]

,5,6,7,8,9,10,11,12,13,14,15,16
87,10,08,06,07,09,05,10,04,NaN,07,07,NaN
175,08,04,06,07,07,06,05,04,06,05,04,04
183,05,03,05,08,08,10,10,09,09,06,NaN,06
204,07,06,05,07,08,10,08,06,NaN,06,07,NaN
206,04,06,04,06,08,09,07,NaN,06,04,07,NaN
246,07,03,05,08,04,08,07,08,05,NaN,07,NaN
302,10,09,10,07,10,10,10,07,05,NaN,NaN,08
392,07,06,08,06,07,01,NaN,07,06,07,08,NaN
548,09,06,09,08,08,07,09,05,NaN,09,07,NaN
990,08,10,10,08,06,04,08,08,10,09,09,NaN


Se define una función que realiza lo siguiente:

- Verfifica si la fila tiene NaN's

- En caso afirmativo localiza los indices donde se encuentran los NaN's

- Si tiene un solo NaN (``if len (ii) < 2``) hace un loop que recorre la fila a partir de 
  dicho indice hasta la penultima posición, y sustituye cada *celda* por el valor de la siguiente
  
- En caso que tenga más de un NaN (``else``) hace lo mismo, pero sustituyendo la celda por el valor de
  de la celda + (número de nan's)

In [24]:
def llenar(x):
    if any(l == True for l in x[:-2].isnull()):
        ii = x[:-2].isnull()
        ii = np.where(ii == 1)[0].tolist()
        if len (ii) < 2:
            for indice in range(ii[0],(len(x)-1)):
                x.iloc[indice] = x.iloc[indice+1]
        else:
            for indice in range(ii[0],(len(x)-len(ii))):
                x.iloc[indice] = x.iloc[indice+len(ii)]
    return x

Se crea un data frame que contega solo los elementos arriba filtrados

In [25]:
prueba = encu_2.iloc[i_nonulos,:]
prueba

,5,6,7,8,9,10,11,12,13,14,15,16
87,10,08,06,07,09,05,10,04,NaN,07,07,NaN
175,08,04,06,07,07,06,05,04,06,05,04,04
183,05,03,05,08,08,10,10,09,09,06,NaN,06
204,07,06,05,07,08,10,08,06,NaN,06,07,NaN
206,04,06,04,06,08,09,07,NaN,06,04,07,NaN
246,07,03,05,08,04,08,07,08,05,NaN,07,NaN
302,10,09,10,07,10,10,10,07,05,NaN,NaN,08
392,07,06,08,06,07,01,NaN,07,06,07,08,NaN
548,09,06,09,08,08,07,09,05,NaN,09,07,NaN
990,08,10,10,08,06,04,08,08,10,09,09,NaN


Se aplica la función anterior a ``prueba``

In [26]:
prueba.apply(llenar, axis =1)

,5,6,7,8,9,10,11,12,13,14,15,16
87,10,08,06,07,09,05,10,04,07,07,NaN,NaN
175,08,04,06,07,07,06,05,04,06,05,04,04
183,05,03,05,08,08,10,10,09,09,06,NaN,06
204,07,06,05,07,08,10,08,06,06,07,NaN,NaN
206,04,06,04,06,08,09,07,06,04,07,NaN,NaN
246,07,03,05,08,04,08,07,08,05,07,NaN,NaN
302,10,09,10,07,10,10,10,07,05,NaN,08,08
392,07,06,08,06,07,01,07,06,07,08,NaN,NaN
548,09,06,09,08,08,07,09,05,09,07,NaN,NaN
990,08,10,10,08,06,04,08,08,10,09,09,NaN


In [27]:
prueba

,5,6,7,8,9,10,11,12,13,14,15,16
87,10,08,06,07,09,05,10,04,07,07,NaN,NaN
175,08,04,06,07,07,06,05,04,06,05,04,04
183,05,03,05,08,08,10,10,09,09,06,NaN,06
204,07,06,05,07,08,10,08,06,06,07,NaN,NaN
206,04,06,04,06,08,09,07,06,04,07,NaN,NaN
246,07,03,05,08,04,08,07,08,05,07,NaN,NaN
302,10,09,10,07,10,10,10,07,05,NaN,08,08
392,07,06,08,06,07,01,07,06,07,08,NaN,NaN
548,09,06,09,08,08,07,09,05,09,07,NaN,NaN
990,08,10,10,08,06,04,08,08,10,09,09,NaN


Sustituimos en el data frame original los valores modificados en el proceso anterior

In [14]:
encu_2.iloc[i_nonulos_p,:] = prueba
encu_2

,5,6,7,8,9,10,11,12,13,14,15,16
0,05,06,04,08,07,03,05,07,07,09,NaN,NaN
1,07,07,06,06,06,08,07,NaN,08,08,NaN,NaN
2,10,10,10,10,10,10,10,10,10,10,NaN,NaN
3,08,07,07,06,07,08,07,NaN,07,08,NaN,NaN
4,09,08,08,08,09,09,08,09,08,08,NaN,NaN
5,07,08,08,06,09,09,05,06,07,09,NaN,NaN
6,06,07,10,10,08,09,10,10,10,09,NaN,NaN
7,08,09,07,09,04,07,09,10,10,09,NaN,NaN
8,07,02,07,08,08,06,09,08,02,07,NaN,NaN
9,09,08,08,07,08,07,09,08,09,09,NaN,NaN


Eliminamos las dos últimas columnas

In [28]:
encu_2.drop(encu_2.columns[-2:],inplace=True,axis=1)

In [29]:
encu_2

,5,6,7,8,9,10,11,12,13,14
0,05,06,04,08,07,03,05,07,07,09
1,07,07,06,06,06,08,07,NaN,08,08
2,10,10,10,10,10,10,10,10,10,10
3,08,07,07,06,07,08,07,NaN,07,08
4,09,08,08,08,09,09,08,09,08,08
5,07,08,08,06,09,09,05,06,07,09
6,06,07,10,10,08,09,10,10,10,09
7,08,09,07,09,04,07,09,10,10,09
8,07,02,07,08,08,06,09,08,02,07
9,09,08,08,07,08,07,09,08,09,09


Ahora procedemos a la siguiente operación, imputar los items vacios por el valor mediano del resto de las valoraciones (fila)

Primero sustituimos los '??' por NAN

In [31]:
encu_3 = encu_2.replace(['??'], np.nan)

In [32]:
encu_3

,5,6,7,8,9,10,11,12,13,14
0,05,06,04,08,07,03,05,07,07,09
1,07,07,06,06,06,08,07,NaN,08,08
2,10,10,10,10,10,10,10,10,10,10
3,08,07,07,06,07,08,07,NaN,07,08
4,09,08,08,08,09,09,08,09,08,08
5,07,08,08,06,09,09,05,06,07,09
6,06,07,10,10,08,09,10,10,10,09
7,08,09,07,09,04,07,09,10,10,09
8,07,02,07,08,08,06,09,08,02,07
9,09,08,08,07,08,07,09,08,09,09


Se define una función que calcula la mediana de cada fila y porteriormente sustituye los nan's por dicho valor

In [33]:
def reemplazo_mediana(x):
    mediana = x.median()
    return x.fillna(mediana)

Se aplica la función anterior a las filas del data frame

In [34]:
encu_3.apply(reemplazo_mediana, axis = 1)

,5,6,7,8,9,10,11,12,13,14
0,05,06,04,08,07,03,05,07,07,09
1,07,07,06,06,06,08,07,7,08,08
2,10,10,10,10,10,10,10,10,10,10
3,08,07,07,06,07,08,07,7,07,08
4,09,08,08,08,09,09,08,09,08,08
5,07,08,08,06,09,09,05,06,07,09
6,06,07,10,10,08,09,10,10,10,09
7,08,09,07,09,04,07,09,10,10,09
8,07,02,07,08,08,06,09,08,02,07
9,09,08,08,07,08,07,09,08,09,09


Ya tenemos los valores correspondientes a los items de valoración *limpios*